# Neo4j->Colab->Bigquery

## 목적 : 빅쿼리에서 json 파일을 잘 인식하지 못하는 것을 확인했는데 neo4j에서 table 형태로 바로 빅쿼리로 데이터를 보내지 못할 경우, 한 번 json 파일을 정리할 필요가 있을 것으로 보여서 만듦

In [ ]:
!pip install neo4j

In [ ]:
import pandas as pd
import json
from pandas.io import gbq
from pandas import DataFrame

# 1. Neo4j 접근하여 바로 DataFrame 만들고 BIgquery로 전달

## 1) Querying Neo4j from Python

In [ ]:
driver = GraphDatabase.driver("neo4j://localhost:7687", auth=("neo4j", "password"))

In [ ]:
with driver.session() as session:
    result = session.run("MATCH (a) RETURN COUNT(a)")
    for r in result:
        print(r)

## 2) Cypher query to DataFrame

In [ ]:
with driver.session() as session:
    result = session.run("MATCH (p:Person)-[:INTERESTED_IN]-(u:Usecase) WITH u.name AS usecase, COUNT(*) AS num RETURN usecase, num")
    df = DataFrame(result.data())

## 3) DataFrame to Bigquery

In [ ]:
df.to_gbq(destination_table='Data_set.data_table', # Bigquery에서 만들어 놓은 Dataset과 Data table 이름 입력
		  project_id='Project_id', # Project id
		  if_exists='replace')

## 4) Bigquery to Python(필요한 경우)

In [ ]:
sql = "SELECT * FROM Data_Set.data_table WHERE * ORDER BY * limit *" # Query문 작성
project_id = 'my_project_id" # Project id
df = pd.read_gbq(sql, project_id=project_id, dialect='standard')

# 2. Neo4j에서 json 파일 받아서 DataFrame 만들고 BIgquery로 전달

## 1) Connect neo4j and python

In [ ]:
driver = GraphDatabase.driver("neo4j://localhost:7687", auth=("neo4j", "password"))
session = driver.session()

In [ ]:
q1="""
CALL apoc.export.json.all("./Data/neo_from_python.json")
"""

In [ ]:
session.run(q1).data()

## 2) Json to DataFrame

In [ ]:
title_lst, year_lst = [], []
with open("./Data/neo_from_python.json", encoding="utf-8") as f:
  data = json.load(f)
# Create 'Title' DataFrame
for i in data :
    json_title = i['title']
    json_date = i['year']
    title_lst.append(json_title)
    year_lst.append(json_date)
df = pd.DataFrame(year_lst, columns = ['year'])
df1 = pd.DataFrame(title_lst, columns = ['title'])
df2 = pd.concat([df,df1],axis=1)

## 3) DataFrame to Bigquery

In [ ]:
df2.to_gbq(destination_table='Data_set.data_table', # Bigquery에서 만들어 놓은 Dataset과 Data table 이름 입력
		  project_id='Project_id', # Project id
		  if_exists='replace')

## 4) Bigquery to Python(필요한 경우)

In [ ]:
sql = "SELECT * FROM Data_Set.data_table WHERE * ORDER BY * limit *" # Query문 작성
project_id = 'my_project_id" # Project id
df = pd.read_gbq(sql, project_id=project_id, dialect='standard')